# LLM Reference Architecture using Redis & Google Cloud Platform

<a href="https://colab.research.google.com/github/RedisVentures/redis-google-llms/blob/main/BigQuery_Palm_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook serves as a getting started guide for working with LLMs on Google Cloud Platform with Redis Enterprise.

## Intro
Google's Vertex AI has expanded its capabilities by introducing [Generative AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview). This advanced technology comes with a specialized [in-console studio experience](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/quickstart), a [dedicated API](https://cloud.google.com/vertex-ai/docs/generative-ai/start/quickstarts/api-quickstart) and [Python SDK](https://cloud.google.com/vertex-ai/docs/python-sdk/use-vertex-ai-python-sdk) designed for deploying and managing instances of Google's powerful Gemini language models.

Redis Enterprise offers robust vector database features, with an efficient API for vector index creation, management, distance metric selection, similarity search, and hybrid filtering. When coupled with its versatile data structures - including lists, hashes, JSON, and sets - Redis Enterprise shines as the optimal solution for crafting high-quality Large Language Model (LLM)-based applications. It embodies a streamlined architecture and exceptional performance, making it an instrumental tool for production environments.

Below we will work through several design patterns with Vertex AI LLMs and Redis Enterprise that will ensure optimal production performance.

___
## Contents
- Setup
    1. Prerequisites
    2. Obtain Dataset
    3. Generate Embeddings
    4. Create Index
    5. Query
- Building a RAG Pipeline from scratch
- Demo

___

# Setup

## 1. Prerequisites
Before we begin, we must install some required libraries, authenticate with Google, create a Redis database, and initialize other required components.

### Install required libraries

In [ ]:
!pip install -U git+https://github.com/RedisVentures/redisvl.git google-cloud-aiplatform langchain unstructured[pdf] gradio
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Using Free Redis Cloud account on GCP
You can also use Forever Free instance of Redis Cloud. To activate it:
- Head to https://redis.com/try-free/
- Register (using gmail-based registration is the easiest)
- Create New Subscription
- Use the following options:
    - Fixed plan, Google Cloud
    - New 30Mb Free database
- Create new RedisStack DB

If you are registering at Redis Cloud for the first time - the last few steps would be performed for you by default. Capture the host, port and default password of the new database. You can use these instead of default `localhost` based in the following code block.

^^^ If prompted press the Restart button to restart the kernel. ^^^

### Install Redis locally (optional)
If you have a Redis db running elsewhere with [Redis Stack](https://redis.io/docs/about/about-stack/) installed, you don't need to run it on this machine. You can skip to the "Connect to Redis server" step.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to Redis server
Replace the connection params below with your own if you are connecting to an external Redis instance.

In [ ]:
import os
import redis

# Redis connection params
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") #"redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      #12110
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  #"pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

# Create Redis client
redis_client = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD
)

# Test connection
redis_client.ping()

True

In [ ]:
# Clear Redis database (optional)
redis_client.flushdb()

True

### Authenticate to Google Cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from getpass import getpass

# input your GCP project ID and region for Vertex AI
PROJECT_ID = getpass("PROJECT_ID:") #'central-beach-194106'
REGION = input("REGION:") #'us-central1'

PROJECT_ID:··········
REGION:us-central1


## 2. Obtain dataset

Below pull the dataset from ...

In [ ]:
#Procure a dataset - downloading a publication from IRS
!mkdir resources
!wget https://www.irs.gov/pub/irs-pdf/p5718.pdf -P resources/

mkdir: cannot create directory ‘resources’: File exists
--2024-04-05 20:04:11--  https://www.irs.gov/pub/irs-pdf/p5718.pdf
Resolving www.irs.gov (www.irs.gov)... 23.213.23.225, 2600:1409:12:2b9::f50, 2600:1409:12:2a4::f50
Connecting to www.irs.gov (www.irs.gov)|23.213.23.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8749823 (8.3M) [application/pdf]
Saving to: ‘resources/p5718.pdf.1’

p5718.pdf.1         100%[===================>]   8.34M  --.-KB/s    in 0.1s    

2024-04-05 20:04:12 (68.8 MB/s) - ‘resources/p5718.pdf.1’ saved [8749823/8749823]



### Create text embeddings with Vertex AI embedding model
Use the [Vertex AI API for text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings), developed by Google.

> Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:
> - **Semantic search**: Search text ranked by semantic similarity.
> - **Recommendation**: Return items with text attributes similar to the given text.
> - **Classification**: Return the class of items whose text attributes are similar to the given text.
> - **Clustering**: Cluster items whose text attributes are similar to the given text.
> - **Outlier Detection**: Return items where text attributes are least related to the given text.

The Vertex AI text-embeddings API lets you create a text embedding using Generative AI on Vertex AI. The `textembedding-gecko` model accepts a maximum of 3,072 input tokens (i.e. words) and outputs 768-dimensional vector embeddings.

### Set up embeddings
We define a helper function to create embeddings from a list of texts convert them to a byte string for efficient storage in Redis.



In [ ]:
from redisvl.utils.vectorize import VertexAITextVectorizer

vectorizer = VertexAITextVectorizer(
    model = "textembedding-gecko@003",
    api_config = {"project_id": PROJECT_ID, "location": REGION}
)

## 3. Generate Embeddings
The next step is to create chunks of the pdf and then embed each chunk as a vector.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

doc = "resources/p5718.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=0
)
loader = UnstructuredFileLoader(
    doc, mode="single", strategy="fast"
)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Done preprocessing. Created 44 chunks of the original pdf resources/p5718.pdf


In [ ]:
# Embed each chunk content
embeddings = vectorizer.embed_many([chunk.page_content for chunk in chunks], as_buffer=True)

# Check to make sure we've created enough embeddings, 1 per document chunk
len(embeddings) == len(chunks)

True

## 4. Create Index

Now that we have created embeddings that represent the text in our dataset, we will create an index that enables efficient search over the embeddings.

**Why do we need to enable search???**
Using Redis for vector search allows us to retrieve chunks of text data that are **similar** or **relevant** to an input question or query. This will be extremely helpful for our sample generative ai / LLM application.

In [ ]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


index_name = "redisvl"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": "chunk"
  },
  "fields": [
    {
        "name": "chunk_id",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "content",
        "type": "text"
    },
    {
        "name": "text_embedding",
        "type": "vector",
        "attrs": {
            "dims": vectorizer.dims,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})

In [ ]:
# Create an index from schema and the client
index = SearchIndex(schema, redis_client)
index.create(overwrite=True, drop=True)

In [ ]:
# Load expects an iterable of dictionaries
data = [
    {
        'chunk_id': f'{i}',
        'content': chunk.page_content,
        'text_embedding': embeddings[i]
    } for i, chunk in enumerate(chunks)
]

# RedisVL handles batching automatically
keys = index.load(data, id_field="chunk_id")

## 5. Query
Now we can use RedisVL to perform a variety of vector search operations.

In [ ]:
from redisvl.query import VectorQuery

query = "What is TCC?"

query_embedding = vectorizer.embed(query)

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=3,
    return_fields=["chunk_id", "content"],
    return_score=True
)

# show the raw redis query
str(vector_query)

'*=>[KNN 3 @text_embedding $vector AS vector_distance] RETURN 3 chunk_id content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3'

In [ ]:
# execute the query with RedisVL
index.query(vector_query)

[{'id': 'chunk:11',
  'vector_distance': '0.343767404556',
  'chunk_id': '11',
  'content': 'When your IRIS Application for TCC is approved and completed, a five-character alphanu- meric TCC that begins with the letter ‘D’ will be assigned to your business. An approval letter will be sent via United States Postal Service (USPS) to the address listed on the application, informing you of your TCC. You can also sign into your IRIS Application for TCC to view your TCCs on the Application Summary page.\n\n13\n\nPublication 5718\n\nIf your application is in Completed status for more than 45 days and your TCC has not been assigned, contact the Help Desk.\n\n1.3.7 Revise Current TCC Information\n\nAs changes occur, you must update and maintain your IRIS TCC Application. Some changes will require all ROs or Authorized Delegates (ADs) on the application to re-sign the Appli- cation Submission page. Below are examples of when an application would need to be re-signed (this list is not all inclusi

In [ ]:
# paginate through results
for result in index.paginate(vector_query, page_size=1):
    print(result[0]["chunk_id"], result[0]["vector_distance"], flush=True)

11 0.343767404556
9 0.344546079636
8 0.345553815365


In [ ]:
from redisvl.query.filter import Text

query = "What is TCC?"

query_embedding = vectorizer.embed(query)

text_filter = Text("content") % "Social Security"

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=3,
    return_fields=["chunk_id", "content"],
    return_score=True,
    filter_expression=text_filter
)

# show the raw redis query
str(vector_query)

'@content:(Social Security)=>[KNN 3 @text_embedding $vector AS vector_distance] RETURN 3 chunk_id content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 3'

In [ ]:
# execute the query with RedisVL
index.query(vector_query)

[{'id': 'chunk:9',
  'vector_distance': '0.344546079636',
  'chunk_id': '9',
  'content': 'Select the role of Transmitter on your application. Note: The TCC for a Transmitter can be used to transmit your own returns and others. You may not use an Issuer TCC to transmit information returns for others.\n\n11\n\nPublication 5718\n\n1.3.3 Third-Party Transmitters\n\nIf you do not have an in-house programmer familiar with XML or do not wish to purchase A2A software that is certified to support the information returns that you plan to file, you can file through a Third-Party Transmitter or use the online Taxpayer Portal. Visit www.irs.gov/ iris for additional information.\n\nOnly those persons listed as an Authorized User on the IRIS Application for TCC qualify to receive information about a Receipt ID associated with a TCC listed on that application.\n\nIf your Third-Party Transmitter needs technical assistance regarding a Receipt ID associated with records that were submitted on behalf of 

# Building a RAG Pipeline from Scratch
We're going to build a complete RAG pipeline from scratch incorporating the following components:

- Standard retrieval and chat completion
- Dense content representation to improve accuracy
- Query re-writing to improve accuracy
- Semantic caching to improve performance
- Conversational session history to improve personalization

In [ ]:
#@title Setup RedisVL *AsyncSearchIndex*

from redis.asyncio import Redis
from redisvl.index import AsyncSearchIndex

# Create Redis client
redis_client = Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)

index = AsyncSearchIndex(index.schema, redis_client)

In [ ]:
#@title Setup VertexAI Generative Model with Safety Settings
from vertexai.generative_models import GenerativeModel, Part, HarmCategory, HarmBlockThreshold


model = GenerativeModel("gemini-1.0-pro-001")

# Define safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

# Define generation config
generation_config = {
    "max_output_tokens": 2048,
    "temperature": 0.5,
    "top_p": 1
}

### Baseline Retrieval Augmented Generation

Below we build a simple RAG pipeline with three helper methods:


*   `answer_question` -- full RAG operation
    * `retrieve_context` -- search Redis for relevant sources
    * `promptify`  -- combine system instructions, user question, and sources



In [ ]:
async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful tax analyst assistant that has access
    to publications from the IRS
    """

    query_vector = vectorizer.embed(query)

    # Fetch context from Redis using vector search
    context = await retrieve_context(index, query_vector)

    prompt = f'''
    System: {SYSTEM_PROMPT}
    User: {promptify(query, context)}
    '''

    responses = model.generate_content(
        [prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False
    )
    # Response provided by LLM
    if(responses.candidates[0].finish_reason.value == 1):
        return(responses.candidates[0].content.parts[0].text)
    else:
        return(f"Content has been blocked for {responses.candidates[0].finish_reason.name} reasons.")


async def retrieve_context(index: AsyncSearchIndex, query_vector) -> str:
    """Fetch the relevant context from Redis using vector search"""
    results = await index.query(
        VectorQuery(
            vector=query_vector,
            vector_field_name="text_embedding",
            return_fields=["content"],
            num_results=3
        )
    )
    content = "\n".join([result["content"] for result in results])
    return content


def promptify(query: str, context: str) -> str:
    return f'''Use the provided context below derived from public documenation to answer the user's question. If you can't answer the user's
    question, based on the context; do not guess. Do your best finding the answer in the context, but if there is no context at all,
    respond with "I don't know".

    User question:

    {query}

    Helpful context:

    {context}

    Answer:
    '''

In [ ]:
# Generate a list of questions
questions = [
    "What is TCC?",
    "Who should apply for an IRIS TCC?",
    "What is a JWK?",
    "Should I buy a yacht??"
]

In [ ]:
import asyncio

results = await asyncio.gather(*[
    answer_question(index, question) for question in questions
])

In [ ]:
for question, result in zip(questions,results):
  print(question+": \n"+result+"\n\n")

What is TCC?: 
TCC stands for Transmitter Control Code.


Who should apply for an IRIS TCC?: 
If you are transmitting information returns to the IRS or if you are developing software to file information returns electronically, you must apply for one or more TCCs using the IRIS Application for TCC available online.


What is a JWK?: 
A JSON Web Key (JWK) is a cryptographic key that is used for e-Services API authentication. It contains a public key that validates the API consumer application. JWKs will have the following criteria:

JWKs should contain a public key using RSA algorithm. RSA provides a key ID for key matching purposes.

Should contain X.509 certificate using both “x5t” (X.509 SHA-1 Thumbprint) and “x5c” (X.509 certificate Chain) parameters.

You are not allowed to use self-signed certificates.

You can use the same public certificate as used for other IRS programs such as MeF or AIR.

For more information on Digital Certificates visit Digital Certificates | Internal Revenu

# Improve performance and cut costs with LLM Semantic Caching

In [ ]:
from redis import Redis
from redisvl.extensions.llmcache import SemanticCache

# Create Redis client
redis_client = Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD
)

# Create the Semantic Cache
llmcache = SemanticCache(
    name="llmcache",
    vectorizer=vectorizer,
    redis_client=redis_client,
    ttl=120,
    distance_threshold=0.2
)

In [ ]:
from functools import wraps


# Create an LLM caching decorator
def cache(func):
    @wraps(func)
    async def wrapper(index, query_text, *args, **kwargs):
        query_vector = llmcache._vectorizer.embed(query_text)

        # Check the cache with the vector
        if result := llmcache.check(vector=query_vector):
            return result[0]['response']

        response = await func(index, query_text, query_vector=query_vector)
        llmcache.store(query_text, response, query_vector)
        return response
    return wrapper


@cache
async def answer_question(index: AsyncSearchIndex, query: str, **kwargs):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful tax analyst assistant that has access
    to publications from the IRS
    """

    # Fetch context from Redis using vector search
    context = await retrieve_context(index, kwargs["query_vector"])

    prompt = f'''
    System: {SYSTEM_PROMPT}
    User: {promptify(query, context)}
    '''

    responses = model.generate_content(
        [prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False
    )
    # Response provided by LLM
    if(responses.candidates[0].finish_reason.value == 1):
        return(responses.candidates[0].content.parts[0].text)
    else:
        return(f"Content has been blocked for {responses.candidates[0].finish_reason.name} reasons.")


In [ ]:
from datetime import datetime

In [ ]:
query = "What is a JWK?"

startTime = datetime.now()
await answer_question(index, query)
print(f"Total time: {datetime.now() - startTime}")

Total time: 0:00:01.661276


In [ ]:
# Now try again with semantic caching enabled!
query = "What's a JWK?"

startTime = datetime.now()
await answer_question(index, query)
print(f"Total time: {datetime.now() - startTime}")

Total time: 0:00:00.110198


In [ ]:
import gradio as gr

async def respond(message, history):
    print(message)
    result = await answer_question(index, message)
    print(result)
    return result

gr.ChatInterface(respond).launch()

20:06:14 httpx INFO   HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
20:06:15 httpx INFO   HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
20:06:16 httpx INFO   HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
20:06:16 httpx INFO   HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

20:06:16 httpx INFO   HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
20:06:16 httpx INFO   HTTP Request: POST https://api.gradio.app/gradio-initiated-analytics/ "HTTP/1.1 200 OK"
20:06:17 httpx INFO   HTTP Request: GET https://api.gradio.app/v2/tunnel-request "HTTP/1.1 200 OK"
20:06:17 httpx INFO   HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-